<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>

### pandas 라이브러리와 탐색적 데이터 분석 과정 익히기

> 다양한 데이터 분석 케이스를 통해 데이터 분석과 pandas 라이브러리 활용에 대해 익히보기로 합니다.

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">탐색적 데이터 분석</font><br>

### 전체 판매 프로세스
1. 해당 쇼핑몰에 중소업체가 계약을 맺고
2. 중소업체가 해당 쇼핑몰에 직접 상품을 올리고
2. 고객이 구매하면, 중소업체가 Olist가 제공하는 물류 파트너를 활용해서 배송을 하고,
3. 고객이 상품을 받으면, 고객에게 이메일 survey 가 전송되고,
4. 고객이 이메일 survey 에 별점과 커멘트를 남겨서 제출하게 됨
    
### 데이터 출처
- 브라질에서 가장 큰 백화점의 이커머스 쇼핑몰 (https://olist.com/)
  - 2016년도부터 2018년도 9월까지의 100k 개의 구매 데이터 정보
  - 구매 상태, 가격, 지불수단, 물류 관련, 리뷰관련, 상품 정보, 구매자 지역 관련 정보

### 주요 질문(탐색하고자 하는 질문 리스트)
- 얼마나 많은 고객이 있는가?
- 고객은 어디에 주로 사는가?
- 고객은 주로 어떤 지불방법을 사용하는가?
- 평균 거래액은 얼마일까?
- **일별, 주별, 월별 판매 트렌드는?**
- 어떤 카테고리가 가장 많은 상품이 팔렸을까?
- 평균 배송 시간은?
</div>

### 기존의 사전 작업

In [ ]:
import pandas as pd
PATH = "00_data/"

In [ ]:
products = pd.read_csv(PATH + "olist_products_dataset.csv", encoding='utf-8-sig')
customers = pd.read_csv(PATH + "olist_customers_dataset.csv", encoding='utf-8-sig')
geolocation = pd.read_csv(PATH + "olist_geolocation_dataset.csv", encoding='utf-8-sig')
order_items = pd.read_csv(PATH + "olist_order_items_dataset.csv", encoding='utf-8-sig')
payments = pd.read_csv(PATH + "olist_order_payments_dataset.csv", encoding='utf-8-sig')
reviews = pd.read_csv(PATH + "olist_order_reviews_dataset.csv", encoding='utf-8-sig')
orders = pd.read_csv(PATH + "olist_orders_dataset.csv", encoding='utf-8-sig')
sellers = pd.read_csv(PATH + "olist_sellers_dataset.csv", encoding='utf-8-sig')
category_name = pd.read_csv(PATH + "product_category_name_translation.csv", encoding='utf-8-sig')

#### orders의 null 인 데이터 삭제하고, payments 와 합치기

In [ ]:
orders = orders.dropna()
merged_order = pd.merge(orders, payments, on='order_id')

In [ ]:
merged_order_payment_date = merged_order[['order_purchase_timestamp', 'payment_value']].copy()

### 1.5. 월별 거래건수 확인하기

In [ ]:
# order_purchase_timestamp 의 날짜 데이터를 기반으로 월별 계산을 해야 하므로 datetime 타입으로 변환
merged_order_payment_date['order_purchase_timestamp'] = pd.to_datetime(merged_order_payment_date['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise') 
merged_order_payment_date = merged_order_payment_date.set_index('order_purchase_timestamp')

merged_order_month_count = merged_order_payment_date.groupby(pd.Grouper(freq='ME')).count() # key 는 기본이 index 임
merged_order_month_count.head()

In [ ]:
import plotly.express as px

# DataFrame의 인덱스(주문 구매 날짜)를 x축, 'payment_value' 열(월별 주문 건수)을 y축 데이터로 사용
fig = px.bar(
    x=merged_order_month_count.index,               # x축: 주문 구매 날짜
    y=merged_order_month_count['payment_value'],      # y축: 주문 건수 (payment_value)
    title="Monthly Order Count",                      # 그래프 제목
    labels={'x': 'Order Purchase Timestamp', 'y': 'Order Count'}  # 축 레이블 지정
)

# x축 라벨이 겹치지 않도록 각도를 조정 (선택 사항)
fig.update_layout(xaxis_tickangle=-45)

# 그래프 출력
fig.show()


### 1.6. 일별 거래액 확인하기

In [ ]:
merged_order_date_sum = merged_order_payment_date.groupby(pd.Grouper(freq='D')).sum() # key 는 기본이 index 임
merged_order_date_sum.head()

In [ ]:
import plotly.express as px

# merged_order_date_sum: 인덱스에 주문 구매 날짜, 'payment_value' 열에 일별 결제 금액 합계가 있음
fig = px.bar(
    x=merged_order_date_sum.index,               # x축: 주문 구매 날짜 (인덱스 사용)
    y=merged_order_date_sum['payment_value'],      # y축: 결제 금액 합계
    title="Daily Payment Value Sum",             # 그래프 제목
    labels={
        'x': 'Order Purchase Timestamp',         # x축 레이블
        'y': 'Payment Value'                       # y축 레이블
    }
)

# x축의 날짜 레이블이 겹치지 않도록 각도를 조정 (선택 사항)
fig.update_layout(xaxis_tickangle=-45)

# 그래프 출력
fig.show()


<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">연습문제</font><br>
일별 거래건수 분석하기<br>
</div>

### 1.7. 시간대별 분석

In [ ]:
merged_order_payment_date = merged_order[['order_purchase_timestamp', 'payment_value']].copy()
merged_order_payment_date.head()

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;"> 팁: pandas 버전별로 지원 기능, 변수/함수명이 변경되는 경우가 있음 </font><br>
- dir() 함수를 통해 지원되는 기능/변수/함수명을 대략적으로 파악할 수 있음 <br>
</div>

In [ ]:
# order_purchase_timestamp 의 날짜 데이터를 기반으로 월별 계산을 해야 하므로 datetime 타입으로 변환
merged_order_payment_date['order_purchase_timestamp'] = pd.to_datetime(merged_order_payment_date['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise') 

merged_order_payment_date.info()

In [ ]:
dir(merged_order_payment_date['order_purchase_timestamp'].dt)

In [ ]:
merged_order_payment_date['order_purchase_timestamp'].dt.quarter

### 사전 설정
- datetime 필드는 dt.시간 별로 필요한 부분만 추출 가능

In [ ]:
merged_order_payment_date['year'] = merged_order_payment_date['order_purchase_timestamp'].dt.year
merged_order_payment_date['monthday'] = merged_order_payment_date['order_purchase_timestamp'].dt.day
merged_order_payment_date['weekday'] = merged_order_payment_date['order_purchase_timestamp'].dt.weekday
merged_order_payment_date['month'] = merged_order_payment_date['order_purchase_timestamp'].dt.month
merged_order_payment_date['hour'] = merged_order_payment_date['order_purchase_timestamp'].dt.hour
merged_order_payment_date['quarter'] = merged_order_payment_date['order_purchase_timestamp'].dt.quarter
merged_order_payment_date['minute'] = merged_order_payment_date['order_purchase_timestamp'].dt.minute

In [ ]:
merged_order_payment_date.head()

### 1.8. 연도별 분석

In [ ]:
merged_order_payment_year = merged_order_payment_date[['year', 'payment_value']].copy()

In [ ]:
merged_order_payment_year

In [ ]:
merged_order_payment_year = merged_order_payment_year.groupby('year').sum()
merged_order_payment_year.head()

In [ ]:
import plotly.express as px

# Plotly Express를 사용하여 막대 그래프 생성
fig = px.bar(
    x=merged_order_payment_year.index,              # x축: 연도 (DataFrame의 인덱스)
    y=merged_order_payment_year['payment_value'],     # y축: 결제 금액 (payment_value 열)
    title="Payment Value by Year",                    # 그래프 제목
    labels={
        'x': 'Year',                                  # x축 레이블
        'y': 'Payment Value'                          # y축 레이블
    }
)

# 그래프 출력
fig.show()


#### 은근히 x tick 제어가 필요할 때가 꽤 있음

In [ ]:
import plotly.express as px

fig = px.bar(
    x=merged_order_payment_year.index,              # x축: 연도 (DataFrame의 인덱스)
    y=merged_order_payment_year['payment_value'],     # y축: 결제 금액 (payment_value 열)
    title="Payment Value by Year",                    # 그래프 제목
    labels={
        'x': 'Year',                                  # x축 레이블
        'y': 'Payment Value'                          # y축 레이블
    }
)

# x축 tick을 [2016, 2017, 2018]로 지정
fig.update_xaxes(tickvals=[2016, 2017, 2018])

# 그래프 출력
fig.show()


### 1.9. 요일별 분석
- weekday : (0:월, 1:화, 2:수, 3:목, 4:금, 5:토, 6:일)

In [ ]:
merged_order_payment_weekday = merged_order_payment_date[['weekday', 'payment_value']].copy()

In [ ]:
merged_order_payment_weekday = merged_order_payment_weekday.groupby('weekday').sum()
merged_order_payment_weekday.head()

In [ ]:
merged_order_payment_weekday = merged_order_payment_weekday.reset_index()

In [ ]:
merged_order_payment_weekday.head()

#### apply() 함수 활용해서 컬럼값 변경하기
- index 를 column 으로 변경하고 (reset_index()), column 값을 순서에 맞춰서 변경 후, index로 재설정(set_index())

In [ ]:
def func(row):
    if row['weekday'] == 0:
        row['weekday'] = 'Mon'  # 월요일
    elif row['weekday'] == 1:
        row['weekday'] = 'Tue'  # 화요일
    elif row['weekday'] == 2:
        row['weekday'] = 'Wed'  # 수요일
    elif row['weekday'] == 3:
        row['weekday'] = 'Thu'  # 목요일
    elif row['weekday'] == 4:
        row['weekday'] = 'Fri'  # 금요일
    elif row['weekday'] == 5:
        row['weekday'] = 'Sat'  # 토요일
    elif row['weekday'] == 6:
        row['weekday'] = 'Sun'  # 일요일
        
    return row

In [ ]:
# 'weekday' 열의 dtype을 object로 변환하여 문자열을 저장할 수 있도록 함
merged_order_payment_weekday['weekday'] = merged_order_payment_weekday['weekday'].astype(object)

# DataFrame의 각 행(row)에 사용자 정의 함수(func)를 적용합니다.
# apply() 함수는 DataFrame의 각 행이나 열을 대상으로 함수를 실행하여,
# 그 결과를 새로운 DataFrame으로 반환합니다.
# - func: 적용할 사용자 정의 함수입니다.
# - axis=1: 함수가 행(row) 단위로 적용되도록 지정합니다.
merged_order_payment_weekday = merged_order_payment_weekday.apply(func, axis=1)
merged_order_payment_weekday

In [ ]:
merged_order_payment_weekday = merged_order_payment_weekday.set_index('weekday')

In [ ]:
merged_order_payment_weekday.head()

In [ ]:
import plotly.express as px

fig = px.bar(
    x=merged_order_payment_weekday.index,         # x축: 요일(weekday)
    y=merged_order_payment_weekday['payment_value'],# y축: 결제 금액 합계
    title="Payment Value by Weekday",               # 그래프 제목
    labels={'x': 'Weekday', 'y': 'Payment Value'}   # 축 레이블 지정
)

# 그래프 출력
fig.show()


### 1.10. 계절별(quarter별) 분석
- quarter : (1:1분기, 2:2분기, 3:3분기, 4:4분기) 

> 물론 현지 사정에 따라 계절 상황을 알면 도움이 됨

In [ ]:
merged_order_payment_quarter = merged_order_payment_date[['quarter', 'payment_value']].copy()

In [ ]:
merged_order_payment_quarter = merged_order_payment_quarter.groupby('quarter').sum()
merged_order_payment_quarter.head()

In [ ]:
merged_order_payment_quarter = merged_order_payment_quarter.reset_index()

In [ ]:
def func(row):
    if row['quarter'] == 1:
        row['quarter'] = '1Q'
    elif row['quarter'] == 2:
        row['quarter'] = '2Q'
    elif row['quarter'] == 3:
        row['quarter'] = '3Q'
    elif row['quarter'] == 4:
        row['quarter'] = '4Q'
        
    return row

In [ ]:
# 'quarter' 컬럼을 문자열 타입으로 변환 (문자열 값을 저장할 수 있도록 함)
merged_order_payment_quarter['quarter'] = merged_order_payment_quarter['quarter'].astype(object)
merged_order_payment_quarter = merged_order_payment_quarter.apply(func, axis=1)

In [ ]:
merged_order_payment_quarter = merged_order_payment_quarter.set_index('quarter')

In [ ]:
merged_order_payment_quarter.head()

In [ ]:
import plotly.express as px

fig = px.bar(
    x=merged_order_payment_quarter.index,             # x축: 분기 (1Q, 2Q, 3Q, 4Q)
    y=merged_order_payment_quarter['payment_value'],    # y축: 결제 금액 합계
    title="Quarterly Payment Value Sum",              # 그래프 제목
    labels={'x': 'Quarter', 'y': 'Payment Value'}       # 축 레이블 지정
)

# 그래프 출력
fig.show()


### 1.11. 시간대별 분석

> 분명히 시간대별 구매율이 높은 구간이 있을 것임

In [ ]:
import pandas as pd
PATH = "00_data/"
products = pd.read_csv(PATH + "olist_products_dataset.csv", encoding='utf-8-sig')
customers = pd.read_csv(PATH + "olist_customers_dataset.csv", encoding='utf-8-sig')
geolocation = pd.read_csv(PATH + "olist_geolocation_dataset.csv", encoding='utf-8-sig')
order_items = pd.read_csv(PATH + "olist_order_items_dataset.csv", encoding='utf-8-sig')
payments = pd.read_csv(PATH + "olist_order_payments_dataset.csv", encoding='utf-8-sig')
reviews = pd.read_csv(PATH + "olist_order_reviews_dataset.csv", encoding='utf-8-sig')
orders = pd.read_csv(PATH + "olist_orders_dataset.csv", encoding='utf-8-sig')
sellers = pd.read_csv(PATH + "olist_sellers_dataset.csv", encoding='utf-8-sig')
category_name = pd.read_csv(PATH + "product_category_name_translation.csv", encoding='utf-8-sig')
orders = orders.dropna()
merged_order = pd.merge(orders, payments, on='order_id')
merged_order_payment_date = merged_order[['order_purchase_timestamp', 'payment_value']].copy()
merged_order_payment_date['order_purchase_timestamp'] = pd.to_datetime(merged_order_payment_date['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise') 
merged_order_payment_date['year'] = merged_order_payment_date['order_purchase_timestamp'].dt.year
merged_order_payment_date['monthday'] = merged_order_payment_date['order_purchase_timestamp'].dt.day
merged_order_payment_date['weekday'] = merged_order_payment_date['order_purchase_timestamp'].dt.weekday
merged_order_payment_date['month'] = merged_order_payment_date['order_purchase_timestamp'].dt.month
merged_order_payment_date['hour'] = merged_order_payment_date['order_purchase_timestamp'].dt.hour
merged_order_payment_date['quarter'] = merged_order_payment_date['order_purchase_timestamp'].dt.quarter
merged_order_payment_date['minute'] = merged_order_payment_date['order_purchase_timestamp'].dt.minute

In [ ]:
merged_order_payment_hour = merged_order_payment_date[['hour', 'payment_value']].copy()

In [ ]:
merged_order_payment_hour = merged_order_payment_hour.groupby('hour').sum()
merged_order_payment_hour.head()

In [ ]:
import plotly.express as px

# Plotly Express를 사용하여 막대 그래프 생성
fig = px.bar(
    x=merged_order_payment_hour.index,               # x축: 시간 (hour) - DataFrame의 인덱스
    y=merged_order_payment_hour['payment_value'],      # y축: 결제 금액 합계 (payment_value 열)
    title="Payment Value by Hour",                     # 그래프 제목
    labels={'x': 'Hour', 'y': 'Payment Value'}         # 축 레이블 지정
)

# x축 라벨 각도 조정 (선택 사항: 긴 레이블의 경우 겹침 방지)
fig.update_layout(xaxis_tickangle=-45)

# 그래프 출력
fig.show()


#### 은근히 x tick 을 제어하고 싶을 때가 있습니다.

In [ ]:
import plotly.express as px

fig = px.bar(
    x=merged_order_payment_hour.index,               # x축: 시간 (hour)
    y=merged_order_payment_hour['payment_value'],      # y축: 결제 금액 합계
    title="Payment Value by Hour",                     # 그래프 제목
    labels={'x': 'Hour', 'y': 'Payment Value'}         # 축 레이블 지정
)

# x축 라벨 각도와 tick 간격을 설정: tick 간격을 1 단위로 지정
fig.update_layout(
    xaxis_tickangle=-45,
    xaxis=dict(dtick=1)
)

fig.show()


### 1.12. 분(minutes)별 분석

In [ ]:
merged_order_payment_minute = merged_order_payment_date[['minute', 'payment_value']].copy()

In [ ]:
merged_order_payment_minute = merged_order_payment_minute.groupby('minute').sum()
merged_order_payment_minute.head()

In [ ]:
import plotly.express as px

# merged_order_payment_minute: 인덱스에 minute, 'payment_value' 열에 값이 있음
fig = px.bar(
    x=merged_order_payment_minute.index,          # x축: minute (인덱스)
    y=merged_order_payment_minute['payment_value'], # y축: payment_value 열의 데이터
    title="Payment Value by Minute",              # 그래프 제목
    labels={'x': 'Minute', 'y': 'Payment Value'}    # x, y축 레이블 지정
)

# 그래프 출력
fig.show()


In [ ]:
import plotly.express as px

fig = px.bar(
    x=merged_order_payment_minute.index,           # x축: minute (인덱스)
    y=merged_order_payment_minute['payment_value'],  # y축: payment_value 열의 데이터
    title="Payment Value by Minute",               # 그래프 제목
    labels={'x': 'Minute', 'y': 'Payment Value'}     # 축 레이블 지정
)

# x축 tick 간격을 1 단위로 설정
fig.update_xaxes(dtick=1)

fig.show()


### 1.13. 요일/시간간 거래액 상관관계 알아보기 
- 상관관계 분석을 위해 사용되는 그래프: heatmap

In [ ]:
merged_order_payment_date.head()

In [ ]:
merged_order_payment_hour_weekday = merged_order_payment_date[['weekday', 'hour', 'payment_value']].copy()

In [ ]:
merged_order_payment_hour_weekday

### weekday 를 요일명으로 변경하기 

- 멀티 인덱스: 하나 이상의 컬럼을 인덱스로 설정할 수 있음

In [ ]:
merged_order_payment_hour_weekday = merged_order_payment_hour_weekday.groupby(['weekday','hour']).sum()

In [ ]:
merged_order_payment_hour_weekday

- 멀티 인덱스에서 특정 인덱스값 가져오기

In [ ]:
merged_order_payment_hour_weekday.index[1][0] # 리스트 안의 리스트일 때처럼 가리키면 됨

In [ ]:
merged_order_payment_hour_weekday = merged_order_payment_hour_weekday.reset_index()

In [ ]:
merged_order_payment_hour_weekday

- apply 함수 만들기

In [ ]:
def func(row):
    if row['weekday'] == 0:
        row['weekday'] = 'Mon'  # 월요일
    elif row['weekday'] == 1:
        row['weekday'] = 'Tue'  # 화요일
    elif row['weekday'] == 2:
        row['weekday'] = 'Wed'  # 수요일
    elif row['weekday'] == 3:
        row['weekday'] = 'Thu'  # 목요일
    elif row['weekday'] == 4:
        row['weekday'] = 'Fri'  # 금요일
    elif row['weekday'] == 5:
        row['weekday'] = 'Sat'  # 토요일
    elif row['weekday'] == 6:
        row['weekday'] = 'Sun'  # 일요일
        
    return row

In [ ]:
# 'weekday' 열의 dtype을 object로 변환하여 문자열을 저장할 수 있도록 함
merged_order_payment_hour_weekday['weekday'] = merged_order_payment_hour_weekday['weekday'].astype(object)
merged_order_payment_hour_weekday = merged_order_payment_hour_weekday.apply(func, axis=1)
merged_order_payment_hour_weekday

### 행 임의 이름 순서로 정렬하기
- 우선 해당 이름에 번호를 매겨 놓은 후, sort_values 로 번호를 기반으로 값으로 정렬함

```python
해당데이터프레임Series = pd.Categorical(시리즈데이터, categories=이름순서리스트, ordered=True)
```

In [ ]:
merged_order_payment_hour_weekday['weekday'] = pd.Categorical(merged_order_payment_hour_weekday['weekday'],categories=['Sun','Mon','Tue','Wed','Thu','Fri','Sat'],ordered=True)

### 행 값으로 정렬하기

```python
sort_values(by=[필드명])
```

In [ ]:
merged_order_payment_hour_weekday = merged_order_payment_hour_weekday.sort_values(by=['weekday', 'hour'], ascending=True)

In [ ]:
merged_order_payment_hour_weekday.head(40)

#### 히트맵 작성
> 막바로 히트맵 작성이 안되는 이유와 이를 해결하기 위해 피벗테이블을 활용하는 방법을 상세히 설명합니다.

##### 1. 원본 데이터의 구조 이해

- **데이터 예시 (`merged_order_payment_hour_weekday.head(40)`):**
  ```plaintext
      weekday   hour   payment_value
  144   Sun      0      38888.45
  145   Sun      1      20137.20
  146   Sun      2      11965.82
  147   Sun      3      6326.13
  148   Sun      4      3914.48
  ```
    - 각 행은 하나의 데이터 포인트를 나타내며, 요일(`weekday`), 시간(`hour`), 그리고 해당 시간대의 결제 금액(`payment_value`)을 포함합니다.
    - 이 형태는 데이터의 관점(행 vs 열)을 명확히 구분하지 않기 때문에, 2차원 배열(grid) 형태로 표현하기에는 적합하지 않습니다.

##### 2. 바로 `go.Heatmap`으로 그리는 코드의 한계

- **예제 코드:**
  ```python
  go.Heatmap(
      z=merged_order_payment_hour_weekday['payment_value'],  # 히트맵에 사용될 데이터 (2D 배열)
      x=merged_order_payment_hour_weekday['hour'],           # x축: 컬럼 레이블
      y=merged_order_payment_hour_weekday['weekday'],        # y축: 인덱스 레이블
  )
  ```
  
- **문제점:**
  - **1D 데이터 전달:**  
    - `z`에 전달된 `merged_order_payment_hour_weekday['payment_value']`는 단일 열(1차원 시리즈)입니다.
    - `go.Heatmap`은 각 셀이 행(요일)과 열(시간)의 교차점에 대응되는 **2차원 배열**을 요구합니다.
  - **x, y의 단순 나열:**  
    - `x`와 `y`도 각각 1차원 시리즈로, 개별 값들만 나열되어 있어 각 값이 어떤 2차원 격자(cell)에 대응되는지 명확하지 않습니다.
  - **결과:**  
    - 위와 같이 long format 데이터를 그대로 사용하면, 히트맵이 올바르게 그려지지 않으며, 데이터 간의 관계(예: 요일별, 시간대별 결제 금액 분포)를 제대로 파악하기 어렵습니다.


##### 3. 피벗 테이블(Pivot Table) 작성의 필요성과 방법

- **피벗 테이블의 목적:**  
  - **데이터 재구조화:**  
    - 데이터를 변환하여, **행**에는 요일(`weekday`), **열**에는 시간(`hour`), **셀 값**에는 결제 금액(`payment_value`)이 배치된 2차원 배열을 만듭니다.
    - 이렇게 재구조화된 데이터를 사용하면, 각 요일과 시간대의 결제 금액이 격자 형태로 표현되어 히트맵에 적합한 형태가 됩니다.

- **피벗 테이블 생성 문법 (예제 기반):**
  ```python
  pivot_df = merged_order_payment_hour_weekday.pivot(
      index='weekday',          # 행: 요일 (예: Sun, Mon, ...)
      columns='hour',           # 열: 시간 (예: 0, 1, 2, ...)
      values='payment_value'    # 셀의 값: 해당 요일과 시간의 결제 금액
  )
  ```
  - **index 인자:** 각 행의 레이블을 `weekday`로 지정합니다.
  - **columns 인자:** 각 열의 레이블을 `hour`로 지정합니다.
  - **values 인자:** 각 셀에 들어갈 값을 `payment_value`로 지정합니다.

- **변환 후 데이터 사용:** 피벗 테이블을 사용하면 데이터가 2차원 배열(`pivot_df.values`)로 변환되며, 각 행(요일)과 열(시간)의 교차점에 해당하는 결제 금액이 올바르게 배치됩니다.
  
##### 4.최종 히트맵 코드

In [ ]:
import plotly.graph_objects as go

# 피벗 테이블 생성: 행은 요일, 열은 시간, 값은 결제 금액
pivot_df = merged_order_payment_hour_weekday.pivot(index='weekday', columns='hour', values='payment_value')

# 히트맵 생성
fig = go.Figure(
    go.Heatmap(
        z=pivot_df.values,
        x=pivot_df.columns,
        y=pivot_df.index
))

fig.update_layout(
    title=dict(text="Heatmap (Plotly Graph Objects)")
)

fig.show()

##### 5.최종 히트맵 개선 코드

In [ ]:
import plotly.graph_objects as go  # Plotly의 그래프 객체 모듈을 임포트합니다.

# 1. 피벗 테이블 생성
# 원본 데이터(merged_order_payment_hour_weekday)를 피벗하여, 행은 요일(weekday), 열은 시간(hour), 셀 값은 결제 금액(payment_value)으로 구성된 2차원 데이터 배열을 생성합니다.
pivot_df = merged_order_payment_hour_weekday.pivot(
    index='weekday',      # 각 행의 레이블: 요일
    columns='hour',       # 각 열의 레이블: 시간
    values='payment_value'  # 각 셀의 값: 결제 금액
)

# 2. 피벗 테이블을 이용해 히트맵 생성
# pivot_df에서 추출한 2차원 배열을 기반으로 히트맵을 만듭니다.
fig = go.Figure(
    data=go.Heatmap(
        z=pivot_df.values,              # 히트맵에 표시될 2차원 데이터 배열 (결제 금액)
        x=pivot_df.columns,             # x축 값: 시간 (hour)
        y=pivot_df.index,               # y축 값: 요일 (weekday)
        zmin=0,                         # 색상 스케일의 최소값을 0으로 설정하여, 데이터 값이 0부터 시작함을 보장
        text=pivot_df.values,           # 각 셀에 표시할 텍스트 데이터 (결제 금액)
        texttemplate='%{z:.0f}'          # 셀 내부 텍스트 포맷: 소수점 없이 정수로 표시
    )
)

# 레이아웃 업데이트: 타이틀과 x축 설정
fig.update_layout(
    title=dict(
        text="Heatmap (Plotly Graph Objects)",  # 그래프의 제목 텍스트
        x=0.5,              # 타이틀을 수평 중앙에 위치 (0~1 사이의 값, 0.5는 중앙)
        y=0.9,             # 타이틀의 수직 위치: 1.0에 가까울수록 그래프 상단에 붙어 표시됩니다.
    ),
    xaxis=dict(
        dtick=1,        # x축 눈금 간격을 1 단위로 설정
        tickformat="d"  # x축 눈금 라벨을 정수 형식으로 표시
    )
)

fig.show()  # 생성한 히트맵을 화면에 출력합니다.

<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>